In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
from pathlib import Path
import pandas as pd
from tqdm import tqdm
import json

In [3]:
from exact_sync.v1.api.annotations_api import AnnotationsApi
from exact_sync.v1.api.images_api import ImagesApi
from exact_sync.v1.api.image_sets_api import ImageSetsApi
from exact_sync.v1.api.annotation_types_api import AnnotationTypesApi
from exact_sync.v1.api.products_api import ProductsApi

from exact_sync.v1.models import ImageSet, Product, AnnotationType, Image, Annotation
from exact_sync.v1.rest import ApiException
from exact_sync.v1.configuration import Configuration
from exact_sync.v1.api_client import ApiClient

## Connect to EXACT

In [6]:
configuration = Configuration()
configuration.username = '######' #'exact'
configuration.password = '#####' #'exact'
configuration.host = "https://euroexact01.euroimmun.de/" #"http://127.0.0.1:8000"

client = ApiClient(configuration)

image_sets_api = ImageSetsApi(client)
annotations_api = AnnotationsApi(client)
annotation_types_api = AnnotationTypesApi(client)
images_api = ImagesApi(client)
product_api = ProductsApi(client)

## Download Rect annotation

In [7]:
annotation_types_points = {}
annotation_types_ref_rect = None

for product in product_api.list_products(name="References").results:
    for annotation_type in annotation_types_api.list_annotation_types(product=product.id, name="BoundingBox").results:
        annotation_types_ref_rect = annotation_type
            
for product in product_api.list_products(name="Reference_Points").results:
    for annotation_type in annotation_types_api.list_annotation_types(product=product.id).results:
        annotation_types_points[annotation_type.name] = annotation_type

annotation_types_points

{'L0': {'area_hit_test': True,
  'closed': True,
  'color_code': '#FFA500',
  'default_height': 50,
  'default_width': 50,
  'enable_blurred': False,
  'enable_concealed': False,
  'id': 11,
  'name': 'L0',
  'node_count': 0,
  'product': 4,
  'sort_order': 2,
  'vector_type': 1},
 'L1': {'area_hit_test': True,
  'closed': True,
  'color_code': '#FFA500',
  'default_height': 50,
  'default_width': 50,
  'enable_blurred': False,
  'enable_concealed': False,
  'id': 12,
  'name': 'L1',
  'node_count': 0,
  'product': 4,
  'sort_order': 3,
  'vector_type': 1},
 'L2': {'area_hit_test': True,
  'closed': True,
  'color_code': '#FFA500',
  'default_height': 50,
  'default_width': 50,
  'enable_blurred': False,
  'enable_concealed': False,
  'id': 13,
  'name': 'L2',
  'node_count': 0,
  'product': 4,
  'sort_order': 4,
  'vector_type': 1},
 'L3': {'area_hit_test': True,
  'closed': True,
  'color_code': '#FFA500',
  'default_height': 50,
  'default_width': 50,
  'enable_blurred': False,
  'e

In [8]:
annotation_types_ref_rect

{'area_hit_test': False,
 'closed': True,
 'color_code': '#800080',
 'default_height': 50000,
 'default_width': 50000,
 'enable_blurred': False,
 'enable_concealed': False,
 'id': 36,
 'name': 'BoundingBox',
 'node_count': 0,
 'product': 3,
 'sort_order': 1,
 'vector_type': 1}

In [13]:
names = [("CRC-A1-5 40X.tif", 5), ("CRC-A1-1 40X.tif", 1), 
         ("CRC-A1-5_stitched.tif", 5), ("CRC-A1-1_2_stitched.tif", 1)]


In [14]:
images_api.list_images(name=names[0][0])

{'count': 1,
 'next': None,
 'previous': None,
 'results': [{'annotations': 'Annotations not load please remove '
                             'omit=annotations',
              'filename': 'CRC-A1-5 40X.tif',
              'height': 105536,
              'id': 20,
              'image_set': 8,
              'image_type': 0,
              'mpp': 0.25,
              'name': 'CRC-A1-5 40X.tif',
              'objective_power': 40.0,
              'time': datetime.datetime(2021, 1, 15, 23, 16, 23, 34132),
              'width': 97840}]}

In [15]:
annotations = []

for name, pair_id  in names:
    
    image = images_api.list_images(name=name).results[0]
    
    for anno in annotations_api.list_annotations(image=image.id, annotation_type=annotation_types_ref_rect.id, # BoundBox ID
                                                 deleted=False, fields="annotation_type,id,image,vector,unique_identifier").results:
        
        annotations.append([image.id, image.name, pair_id, anno.id, anno.vector, anno.unique_identifier, anno.annotation_type])
        
annotations = pd.DataFrame(annotations, columns=["image_id", "image_name", "pair", "id", "vector", "unique_identifier", "annotation_type"])
annotations.head()

,image_id,image_name,pair,id,vector,unique_identifier,annotation_type
0,20,CRC-A1-5 40X.tif,5,30841,"{'x1': 652, 'x2': 71998, 'y1': 6290, 'y2': 105...",e3d05bf7-a2db-4bea-9b5b-2ea7038e2d7b,36
1,19,CRC-A1-1 40X.tif,1,30843,"{'x1': 225, 'x2': 70248, 'y1': 421, 'y2': 77413}",cdee5cc7-f5ce-481c-a483-689526d5d02c,36
2,10,CRC-A1-5_stitched.tif,5,30840,"{'x1': 14906, 'x2': 143453, 'y1': 7915, 'y2': ...",36665283-c06e-442d-bd4e-893e8ccaec9e,36
3,11,CRC-A1-1_2_stitched.tif,1,30842,"{'x1': 23026, 'x2': 147549, 'y1': 1432, 'y2': ...",12e73dc1-72eb-484f-b46d-b4c03d7e0cbc,36


In [17]:
num_lines = 5
patch_size = 100

registration_points = []
for id, bounding_box in annotations.iterrows():
    
    box_width = bounding_box.vector["x2"] - bounding_box.vector["x1"]
    box_height = bounding_box.vector["y2"] - bounding_box.vector["y1"]
    
    step_size_x = box_width // num_lines
    step_size_y = box_height // num_lines
        
    point_index = 0
    for x in range(bounding_box.vector["x1"] + step_size_x //2, bounding_box.vector["x2"], step_size_x):
        for y in range(bounding_box.vector["y1"] + step_size_y //2, bounding_box.vector["y2"], step_size_y):
            
            ref_point = annotation_types_points[f"L{point_index}"]
                
            vector = {"x1": x - patch_size, "y1": y - patch_size, 
                          "x2": x + patch_size, "y2": y + patch_size}
                
            point = [bounding_box.image_id, vector, point_index, ref_point.name, ref_point.id]
                
            registration_points.append(point)
                
            point_index += 1
    
registration_points = pd.DataFrame(registration_points, 
                                   columns=["image_id", "vector", "point_index", "point_name", "point_id"])
registration_points

,image_id,vector,point_index,point_name,point_id
0,20,"{'x1': 7686, 'y1': 16069, 'x2': 7886, 'y2': 16...",0,L0,11
1,20,"{'x1': 7686, 'y1': 35827, 'x2': 7886, 'y2': 36...",1,L1,12
2,20,"{'x1': 7686, 'y1': 55585, 'x2': 7886, 'y2': 55...",2,L2,13
3,20,"{'x1': 7686, 'y1': 75343, 'x2': 7886, 'y2': 75...",3,L3,14
4,20,"{'x1': 7686, 'y1': 95101, 'x2': 7886, 'y2': 95...",4,L4,15
...,...,...,...,...,...
95,11,"{'x1': 134994, 'y1': 15674, 'x2': 135194, 'y2'...",20,L20,31
96,11,"{'x1': 134994, 'y1': 44359, 'x2': 135194, 'y2'...",21,L21,32
97,11,"{'x1': 134994, 'y1': 73044, 'x2': 135194, 'y2'...",22,L22,33
98,11,"{'x1': 134994, 'y1': 101729, 'x2': 135194, 'y2...",23,L23,34


## Upload ref Points to EXACT

In [18]:
annotations_to_upload = []

for id, bounding_box in tqdm(registration_points.iterrows()):
    
    
    exact_annos =  annotations_api.list_annotations(image=bounding_box.image_id, annotation_type=bounding_box.point_id).results
    
    if len(exact_annos) == 0:
        anno = Annotation(annotation_type=bounding_box.point_id, 
                          vector=bounding_box.vector, 
                          image=bounding_box.image_id)

        annotations_to_upload.append(anno)

    else:
        prinf(f"ImageId: {bounding_box.image_id}, AnnotationType: {bounding_box.point_id} combination allready exists!")

100it [00:18,  5.42it/s]


In [19]:
annotations_to_upload[:3]

[{'annotation_type': 11,
  'annotationversion_set': [],
  'blurred': None,
  'concealed': None,
  'deleted': None,
  'description': None,
  'id': None,
  'image': 20,
  'last_edit_time': None,
  'last_editor': None,
  'meta_data': None,
  'time': None,
  'unique_identifier': None,
  'uploaded_media_files': [],
  'user': None,
  'vector': {'x1': 7686, 'x2': 7886, 'y1': 16069, 'y2': 16269},
  'verified_by_user': None},
 {'annotation_type': 12,
  'annotationversion_set': [],
  'blurred': None,
  'concealed': None,
  'deleted': None,
  'description': None,
  'id': None,
  'image': 20,
  'last_edit_time': None,
  'last_editor': None,
  'meta_data': None,
  'time': None,
  'unique_identifier': None,
  'uploaded_media_files': [],
  'user': None,
  'vector': {'x1': 7686, 'x2': 7886, 'y1': 35827, 'y2': 36027},
  'verified_by_user': None},
 {'annotation_type': 13,
  'annotationversion_set': [],
  'blurred': None,
  'concealed': None,
  'deleted': None,
  'description': None,
  'id': None,
  'ima

## Check that no error occurred and afterwards upload annotations

In [20]:
#annotations_api.create_annotation(body=annotations_to_upload)

[{'annotation_type': 11,
  'annotationversion_set': [],
  'blurred': 'False',
  'concealed': 'False',
  'deleted': False,
  'description': '',
  'id': 30844,
  'image': 20,
  'last_edit_time': datetime.datetime(2021, 1, 16, 10, 44, 18, 519762),
  'last_editor': 2,
  'meta_data': None,
  'time': datetime.datetime(2021, 1, 16, 10, 44, 18, 519735),
  'unique_identifier': 'a8c564e8-fd69-43b7-b9a5-f1f16c7bbbd5',
  'uploaded_media_files': [],
  'user': 2,
  'vector': {'x1': 7686, 'x2': 7886, 'y1': 16069, 'y2': 16269},
  'verified_by_user': 'False'},
 {'annotation_type': 12,
  'annotationversion_set': [],
  'blurred': 'False',
  'concealed': 'False',
  'deleted': False,
  'description': '',
  'id': 30845,
  'image': 20,
  'last_edit_time': datetime.datetime(2021, 1, 16, 10, 44, 18, 519815),
  'last_editor': 2,
  'meta_data': None,
  'time': datetime.datetime(2021, 1, 16, 10, 44, 18, 519806),
  'unique_identifier': '9a8e6a0c-0a43-4f6c-b6d5-4eeeca8b0dcc',
  'uploaded_media_files': [],
  'user':